In [1]:
from planet import api
from planet.api import filters
from sys import stdout
import urllib.request
import shutil
from requests.auth import HTTPBasicAuth
import os
import time
import requests
import cv2
import glob
import subprocess
import rasterio
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import wikipedia
from xml.dom import minidom

In [2]:
title = input("Please input your volcano of interest: ")

Please input your volcano of interest: Anak Krakatoa


In [3]:
page = wikipedia.WikipediaPage(title)

In [4]:
print("Wikipedia summary for your volcano of interest: ")
print("")
print(wikipedia.summary(title))

Wikipedia summary for your volcano of interest: 

Anak Krakatoa, (English: Child of Krakatoa) is an island in a caldera in the Sunda Strait between the islands of Java and Sumatra in the Indonesian province of Lampung. On 29 December 1927, Anak Krakatau, emerged from the caldera formed in 1883 by the explosive volcanic eruption which destroyed the island of Krakatoa. There has been sporadic eruptive activity at the site since the late 20th century, culminating with a large underwater collapse of the volcano which caused a deadly tsunami in December 2018, followed by subsequent activity in 2019. Due to its young age, the island is one of several in the area which are of considerable interest to volcanologists, and the subject of extensive study.


In [5]:
print("References from the wikipedia article of your volcano of interest: ")
print("")

References_Wikipedia = page.references

len(References_Wikipedia)
x = 1

for i in References_Wikipedia:
    print('[',x,'] :',i)
    x = x + 1

References from the wikipedia article of your volcano of interest: 

[ 1 ] : http://in.reuters.com/article/worldNews/idINIndia-30436520071111?sp=true
[ 2 ] : http://portal.vsi.esdm.go.id/joomla/
[ 3 ] : http://doi.org/10.1144%2FSP361.7
[ 4 ] : http://sp.lyellcollection.org/content/361/1/79
[ 5 ] : http://tools.wmflabs.org/geohack/geohack.php?pagename=Anak_Krakatoa&params=6.102_S_105.423_E_type:mountain_scale:100000
[ 6 ] : http://www.worldcat.org/issn/0305-8719
[ 7 ] : https://www.bbc.com/news/science-environment-46707731
[ 8 ] : https://www.nbcnews.com/news/world/indonesia-s-anak-krakatau-volcano-triggered-deadly-tsunami-now-quarter-n952906
[ 9 ] : https://www.rt.com/news/441477-tour-boat-fire-bombed-volcano/
[ 10 ] : https://www.volcanodiscovery.com/krakatau/news/80657/Krakatau-volcano-Indonesia-activity-update-and-field-report-increasing-unrest.html
[ 11 ] : https://www.volcanodiscovery.com/krakatoa/2018/dec/eruption-tsunami/updates.html
[ 12 ] : https://volcano.si.edu/volcano.cfm?v

In [6]:
print("Related wikipedia articles to your volcano of interest: ")
print("")

Related_Wikipedia = wikipedia.geosearch(page.coordinates[0], page.coordinates[1])

len(Related_Wikipedia)
x = 1

for i in Related_Wikipedia:
    print ('[',x,'] :', i, '-', wikipedia.page(i).url)
    x = x + 1

Related wikipedia articles to your volcano of interest: 

[ 1 ] : Krakatoa archipelago - https://en.wikipedia.org/wiki/Krakatoa_archipelago
[ 2 ] : Anak Krakatoa - https://en.wikipedia.org/wiki/Anak_Krakatoa
[ 3 ] : Krakatoa - https://en.wikipedia.org/wiki/Krakatoa
[ 4 ] : 1883 eruption of Krakatoa - https://en.wikipedia.org/wiki/1883_eruption_of_Krakatoa
[ 5 ] : 2018 Sunda Strait tsunami - https://en.wikipedia.org/wiki/2018_Sunda_Strait_tsunami


In [7]:
lat = float(page.coordinates[0])
long = float(page.coordinates[1])

In [8]:
start_date = input("Please input your start date (YYYY-MM-DD): ")

Please input your start date (YYYY-MM-DD): 2018-12-01


In [9]:
end_date = input("Please input your end date (YYYY-MM-DD): ")

Please input your end date (YYYY-MM-DD): 2019-01-31


In [10]:
cloud_cover = input("Please input the percent of maximum cloud cover you want to allow in each image as an integer between 0 and 1 (e.g. 20% = 0.2): ")
cloud_cover = float(cloud_cover)

Please input the percent of maximum cloud cover you want to allow in each image as an integer between 0 and 1 (e.g. 20% = 0.2): 0.2


In [11]:
api_key = input("Please input you Planet API key: ") 

Please input you Planet API key: fa8bc840c2af4b52a4f6a69e3489fd10


In [12]:
client = api.ClientV1(api_key) 
PL_API_KEY = api_key

In [13]:
geo_json_geometry = {
  "type": "Polygon",
        "coordinates": [
          [
            [
              long - 0.07,
              lat + 0.07
            ],
            [
              long + 0.07,
              lat + 0.07
            ],
            [
              long + 0.07,
              lat - 0.07
            ],
            [
              long - 0.07,
              lat - 0.07
            ],
            [
              long - 0.07,
              lat + 0.07
            ]
          ]
        ]
      }

In [14]:
geometry_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": geo_json_geometry
}

In [15]:
date_range_filter = {
    "type": "DateRangeFilter",
    "field_name": "acquired",
    "config": {
        "gte": start_date + "T00:00:00.000Z",
        "lte": end_date + "T00:00:00.000Z"
    }
}

In [16]:
cloud_cover_filter = {
    "type": "RangeFilter",
    "field_name": "cloud_cover",
    "config": {
        "lte": cloud_cover,
    }
}

In [17]:
item_types = ['PSScene4Band']
asset_type = 'analytic'

In [18]:
query = {
    "type": "AndFilter",
    "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [19]:
request = filters.build_search_request(
    query, item_types
)

In [20]:
dataset = client.quick_search(request)

stdout.write('id,cloud_cover,date\n')

id,cloud_cover,date


In [21]:
for item in dataset.items_iter(limit=None):
    props = item['properties']
    stdout.write('{0},{cloud_cover},{acquired}\n'.format(item['id'], **props))

20190129_030555_1053,0.18,2019-01-29T03:05:55.637413Z
20190102_031011_0f2d,0.2,2019-01-02T03:10:11.888407Z
20190126_030500_1051,0.13,2019-01-26T03:05:00.817577Z
20190128_025336_1034,0.13,2019-01-28T02:53:36.276178Z
20190129_025214_101b,0.1,2019-01-29T02:52:14.358321Z
20190124_025423_1040,0,2019-01-24T02:54:23.125827Z
20190112_025426_1018,0.02,2019-01-12T02:54:26.28183Z
20190112_025425_1018,0.01,2019-01-12T02:54:25.23273Z
20190112_025424_1018,0,2019-01-12T02:54:24.18363Z
20190113_025310_0f42,0.01,2019-01-13T02:53:10.998835Z
20190113_025309_0f42,0.15,2019-01-13T02:53:09.924045Z
20190107_025219_1027,0.14,2019-01-07T02:52:19.741871Z
20190107_025218_1027,0.03,2019-01-07T02:52:18.69382Z
20190107_025217_1027,0.07,2019-01-07T02:52:17.645768Z
20190107_025220_1027,0.01,2019-01-07T02:52:20.789923Z
20190106_030927_0f33,0.1,2019-01-06T03:09:27.609053Z
20190104_025219_101b,0.06,2019-01-04T02:52:19.043151Z
20190104_025217_101b,0.17,2019-01-04T02:52:17.993527Z
20190104_025215_101b,0.13,2019-01-04T02:5

In [22]:
for i in item_types:
    request = api.filters.build_search_request(query, [i])  # ,name = None, interval= 'day')
    dataset = client.quick_search(request)
    print(i)
    for item in dataset.items_iter(limit=100):
        print(item['id'])
        session = requests.Session()
        session.auth = (PL_API_KEY, '')
        results = \
            session.get(
                ("https://api.planet.com/data/v1/item-types/" +
                 "{}/items/{}/assets/").format(i, item['id']))
       
        item_activation_url = results.json()[asset_type]["_links"]["activate"]
       
        response = session.post(item_activation_url)
        print(response.status_code)
        while response.status_code != 204:
            time.sleep(15)
            response = session.post(item_activation_url)
            response.status_code = response.status_code
            print(response.status_code)
        
        item_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(i, item['id'])
        result = requests.get(item_url, auth=HTTPBasicAuth(PL_API_KEY, ''))
        download_url = result.json()[asset_type]['location']  # KeyError: 'location'
        output_file = item['id'] + '.tif'

        with urllib.request.urlopen(download_url) as response, open(output_file, 'wb') as out_file:
            shutil.copyfileobj(response, out_file)

PSScene4Band
20190129_030555_1053
204


KeyboardInterrupt: 

In [ ]:
item_types = ['PSScene4Band']
asset_type = 'analytic_xml'

In [ ]:
query = {
    "type": "AndFilter",
    "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [ ]:
request = filters.build_search_request(
    query, item_types
)

In [ ]:
dataset = client.quick_search(request)

stdout.write('id,cloud_cover,date\n')

In [ ]:
for item in dataset.items_iter(limit=None):
    props = item['properties']
    stdout.write('{0},{cloud_cover},{acquired}\n'.format(item['id'], **props))

In [ ]:
for i in item_types:
    request = api.filters.build_search_request(query, [i])  # ,name = None, interval= 'day')
    dataset = client.quick_search(request)
    print(i)
    for item in dataset.items_iter(limit=50):
        print(item['id'])
        session = requests.Session()
        session.auth = (PL_API_KEY, '')
        results = \
            session.get(
                ("https://api.planet.com/data/v1/item-types/" +
                 "{}/items/{}/assets/").format(i, item['id']))
       
        item_activation_url = results.json()[asset_type]["_links"]["activate"]
       
        response = session.post(item_activation_url)
        print(response.status_code)
        while response.status_code != 204:
            time.sleep(15)
            response = session.post(item_activation_url)
            response.status_code = response.status_code
            print(response.status_code)
        
        item_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(i, item['id'])
        result = requests.get(item_url, auth=HTTPBasicAuth(PL_API_KEY, ''))
        download_url = result.json()[asset_type]['location']  # KeyError: 'location'
        output_file = item['id'] + '.xml'

        with urllib.request.urlopen(download_url) as response, open(output_file, 'wb') as out_file:
            shutil.copyfileobj(response, out_file)

In [ ]:
tifs = []
xmls = []

for r, d, f in os.walk(os.getcwd()):
    for file in f:
        if file.endswith('.tif'):
            tifs.append(os.path.join(r, file))
        if file.endswith('.xml'):
            xmls.append(os.path.join(r, file))
            
i = 0
        
for x in xmls:
    with rasterio.open(tifs[i][-24:]) as src:
            band_blue = src.read(1, masked = True)    
    with rasterio.open(tifs[i][-24:]) as src:
            band_green = src.read(2, masked = True)
    with rasterio.open(tifs[i][-24:]) as src:
            band_red = src.read(3, masked = True)
    with rasterio.open(tifs[i][-24:]) as src:
            band_nir = src.read(4, masked = True)
    xmldoc = minidom.parse(xmls[i][-24:])
    nodes = xmldoc.getElementsByTagName("ps:bandSpecificMetadata")
    
    coeffs = {}
    for node in nodes:
        bn = node.getElementsByTagName("ps:bandNumber")[0].firstChild.data
        if bn in ['1', '2', '3', '4']:
            a = int(bn)
            value = node.getElementsByTagName('ps:reflectanceCoefficient')[0].firstChild.data
            coeffs[a] = float(value)
    
    band_blue = band_blue * coeffs[1]
    band_green = band_green * coeffs[2]
    band_red = band_red * coeffs[3]
    band_nir = band_nir * coeffs[4]
    
    np.seterr(divide='ignore', invalid='ignore')
    ndvi = ((band_nir.astype(float) - band_red.astype(float)) / (band_nir + band_red))
    
    kwargs = src.meta
    kwargs.update(
    dtype = rasterio.float32,
    count = 1)
    
    with rasterio.open('ndvi_' + tifs[i][-24:], 'w', **kwargs) as dst:
        dst.write_band(1, ndvi.astype(rasterio.float32))
    
        
    i = i + 1

In [ ]:
os.getcwd()
os.mkdir('Vegetation_Indices_Imagery')

In [ ]:
sp0 = os.getcwd()
sf0 = os.listdir(sp0)
dp0 = sp0 + ('\\Vegetation_Indices_Imagery')
for file in sf0:
    if file.startswith(('ndvi')):
        shutil.move(os.path.join(sp0,file), os.path.join(dp0, file))

In [ ]:
images = [f for f in os.listdir(dp0) if os.path.isfile(os.path.join(dp0, f))]

for image in images:
    folder_name = image.split('_')[1]

    new_path = os.path.join(dp0, folder_name)
    if not os.path.exists(new_path):
        os.makedirs(new_path)

    old_image_path = os.path.join(dp0, image)
    new_image_path = os.path.join(new_path, image)
    shutil.move(old_image_path, new_image_path)

In [ ]:
os.getcwd()
os.mkdir('Original_Imagery')

In [ ]:
sp = os.getcwd()
sf = os.listdir(sp)
dp = sp + ('\\Original_Imagery')
for file in sf:
    if file.endswith(('.tif','.xml')):
        shutil.move(os.path.join(sp,file), os.path.join(dp, file))

In [ ]:
images = [f for f in os.listdir(dp) if os.path.isfile(os.path.join(dp, f))]

for image in images:
    folder_name = image.split('_')[0]

    new_path = os.path.join(dp, folder_name)
    if not os.path.exists(new_path):
        os.makedirs(new_path)

    old_image_path = os.path.join(dp, image)
    new_image_path = os.path.join(new_path, image)
    shutil.move(old_image_path, new_image_path)

In [ ]:
folders = []
for r, d, f in os.walk(dp0):
    for folder in d:
        folders.append(os.path.join(r, folder))

i = 0
for x in folders:
    files_to_mosaic = glob.glob(folders[i] + '/*.tif')
    files_string = " ".join(files_to_mosaic)
    command = "python " + sp + "\gdal_merge.py -o " + files_string[-24:-16] + ".tif" + " -of gtiff " + files_string
    print(command)
    i = i + 1
    output = subprocess.getoutput(command)
    print(output)

In [ ]:
os.mkdir('Merged_Tifs')

In [ ]:
sp2 = os.getcwd()
sf2 = os.listdir(sp2)
dp2 = sp2 + ('\\Merged_Tifs')
for file in sf2:
    if file.endswith('.tif'):
        shutil.move(os.path.join(sp2,file), os.path.join(dp2, file))

In [ ]:
os.chdir(sp2 + '\\Merged_Tifs')

In [ ]:
files = []
for r, d, f in os.walk(dp2):
    for file in f:
        files.append(os.path.join(r, file))
        
timeavg = []
ndviavg = []

i = 0
for x in files:
    with rasterio.open(files[i]) as src:
            ndvi = src.read(1, masked = True)
    
    ndvi2 = np.ma.masked_where(ndvi < 0.01, ndvi)
    
    fig2 = plt.figure(figsize=(10,10))
    ax = fig2.add_subplot(111)
    
    plt.title(f[i] + " NDVI Histogram", fontsize=18, fontweight='bold')
    plt.xlabel("NDVI values", fontsize=14)
    plt.ylabel("# pixels", fontsize=14)
    
    b = ndvi2[~np.isnan(ndvi2)]
    numBins = 20
    ax.hist(b,numBins,color='blue',alpha=0.8)
    ax.set_xlim([-1, 1])
    fig2.savefig("ndvi_" + f[i][0:8] + "_histogram.png", dpi=200, bbox_inches='tight', pad_inches=0.7)

    
    ndvi_adjusted = ndvi * (-255.0)
    masked_ndvi_adjusted = np.ma.array(ndvi_adjusted, mask = (ndvi_adjusted == 0.))
    cmap = plt.cm.RdYlGn
    cmap.set_bad(color = 'white',alpha = 0.0)
    img = masked_ndvi_adjusted.astype(np.uint8)
    plt.imsave("ndvi_" + f[i][0:8] + "_colormap.tiff", img, cmap=cmap)
    plt.close()
    
    avg = np.average(b)
    ndviavg.append(avg)
    dates = f[i][4:8]
    timeavg.append(dates)
       
    i = i + 1
    
fig3 = plt.figure(figsize=(10,10))
ax2 = fig3.add_subplot(111)
ax2.set_ylim([0, 1])
plt.plot(timeavg, ndviavg, color = 'blue', linewidth = 4)
plt.axhspan(0, 0.3, facecolor='r', alpha = 0.5, label = 'Unhealthy')
plt.axhspan(0.3, 0.6, facecolor='y', alpha = 0.5, label = 'Somewhat Healthy')
plt.axhspan(0.6, 1, facecolor='g', alpha = 0.5, label = 'Healthy')
plt.legend()
plt.title("Average NDVI Over Time", fontsize=18, fontweight='bold')
plt.xlabel("Start Date & End Date", fontsize=14)
plt.ylabel("Average NDVI", fontsize=14)
fig3.savefig("AverageNDVI_OverTime.png", dpi=200, bbox_inches='tight', pad_inches=0.7)
plt.close()

In [ ]:
sp3 = os.getcwd()
sf3 = os.listdir(sp3)

for file in sf3:
    if file.startswith('ndvi'):
        shutil.move(os.path.join(sp3,file), os.path.join(sp2,file))

In [ ]:
os.chdir(sp2)
os.mkdir('Vegetation_Indices')

In [ ]:
sp4 = os.getcwd()
sf4 = os.listdir(sp4)
dp4 = sp2 + '\\Vegetation_Indices'

for file in sf3:
    if file.startswith('ndvi'):
        shutil.move(os.path.join(sp4,file), os.path.join(dp4,file))

In [ ]:
os.chdir('Merged_Tifs')

In [ ]:
sp3 = os.getcwd()
sf3 = os.listdir(sp3)

for file in sf3:
    if file.startswith('Ave'):
        shutil.move(os.path.join(sp3,file), os.path.join(sp2,file))

In [ ]:
os.chdir(sp2)
os.mkdir('Average_NDVI')

In [ ]:
sp4 = os.getcwd()
sf4 = os.listdir(sp4)
dp4 = sp2 + '\\Average_NDVI'

for file in sf3:
    if file.startswith('Ave'):
        shutil.move(os.path.join(sp4,file), os.path.join(dp4,file))

In [ ]:
os.chdir(sp2)

In [ ]:
item_types = ['PSScene3Band']
asset_type = 'visual'

In [ ]:
query = {
    "type": "AndFilter",
    "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [ ]:
request = filters.build_search_request(
    query, item_types
)

In [ ]:
dataset = client.quick_search(request)

stdout.write('id,cloud_cover,date\n')

In [ ]:
for item in dataset.items_iter(limit=None):
    props = item['properties']
    stdout.write('{0},{cloud_cover},{acquired}\n'.format(item['id'], **props))

In [ ]:
for i in item_types:
    request = api.filters.build_search_request(query, [i])  # ,name = None, interval= 'day')
    dataset = client.quick_search(request)
    print(i)
    for item in dataset.items_iter(limit=100):
        print(item['id'])
        session = requests.Session()
        session.auth = (PL_API_KEY, '')
        results = \
            session.get(
                ("https://api.planet.com/data/v1/item-types/" +
                 "{}/items/{}/assets/").format(i, item['id']))
       
        item_activation_url = results.json()[asset_type]["_links"]["activate"]
       
        response = session.post(item_activation_url)
        print(response.status_code)
        while response.status_code != 204:
            time.sleep(15)
            response = session.post(item_activation_url)
            response.status_code = response.status_code
            print(response.status_code)
        
        item_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(i, item['id'])
        result = requests.get(item_url, auth=HTTPBasicAuth(PL_API_KEY, ''))
        download_url = result.json()[asset_type]['location']  # KeyError: 'location'
        output_file = item['id'] + '.tif'

        with urllib.request.urlopen(download_url) as response, open(output_file, 'wb') as out_file:
            shutil.copyfileobj(response, out_file)

In [ ]:
os.mkdir('Original_Ash_Imagery')

In [ ]:
sp5 = os.getcwd()
sf5 = os.listdir(sp5)
dp5 = sp5 + ('\\Original_Ash_Imagery')
for file in sf5:
    if file.endswith('.tif'):
        shutil.move(os.path.join(sp5,file), os.path.join(dp5, file))

In [ ]:
images = [f for f in os.listdir(dp5) if os.path.isfile(os.path.join(dp5, f))]

for image in images:
    folder_name = image.split('_')[0]

    new_path = os.path.join(dp5, folder_name)
    if not os.path.exists(new_path):
        os.makedirs(new_path)

    old_image_path = os.path.join(dp5, image)
    new_image_path = os.path.join(new_path, image)
    shutil.move(old_image_path, new_image_path)

In [ ]:
folders = []
for r, d, f in os.walk(dp5):
    for folder in d:
        folders.append(os.path.join(r, folder))

i = 0
for x in folders:
    files_to_mosaic = glob.glob(folders[i] + '/*.tif')
    files_string = " ".join(files_to_mosaic)
    command = "python " + sp5 + "\gdal_merge.py -o "+ files_string[-33:-25] + ".tif" + " -of gtiff " + files_string
    print(command)
    i = i + 1
    output = subprocess.getoutput(command)
    print(output)

In [ ]:
os.mkdir('Merged_Ash_Tifs')

In [ ]:
sp6 = os.getcwd()
sf6 = os.listdir(sp5)
dp6 = sp6 + ('\\Merged_Ash_Tifs')
for file in sf6:
    if file.endswith('.tif'):
        shutil.move(os.path.join(sp6,file), os.path.join(dp6, file))

In [ ]:
os.chdir(dp6)

In [ ]:
flags = [i for i in dir(cv2) if i.startswith('COLOR')]

In [ ]:
range1_lo = (140, 130, 105, 255)
range1_hi = (166, 152, 129, 255)

range2_lo = (201, 190, 175, 255)
range2_hi = (210, 200, 190, 255)

range3_lo = (121, 118, 114, 255)
range3_hi = (124, 123, 119, 255)

range4_lo = (145, 142, 130, 255)
range4_hi = (150, 147, 135, 255)

range5_lo = (165, 165, 150, 255)
range5_hi = (195, 185, 175, 255)

range6_lo = (155, 150, 140, 255)
range6_hi = (170, 160, 148, 255)

range7_lo = (127, 126, 115, 255)
range7_hi = (135, 128, 125, 255)

range8_lo = (190, 155, 130, 255)
range8_hi = (230, 195, 165, 255)

In [ ]:
files = []
for r, d, f in os.walk(dp6):
    for file in f:
        files.append(os.path.join(r, file))

ash_pixels = []
timeavg = []

i = 0
for x in files:
    img = cv2.imread(files[i])
    img2 = cv2.cvtColor(img, cv2.COLOR_BGRA2RGBA)
    
    mask_range1 = cv2.inRange(img2, range1_lo, range1_hi)
    mask_range2 = cv2.inRange(img2, range2_lo, range2_hi)
    mask_range3 = cv2.inRange(img2, range3_lo, range3_hi)
    mask_range4 = cv2.inRange(img2, range4_lo, range4_hi)
    mask_range5 = cv2.inRange(img2, range5_lo, range5_hi)
    mask_range6 = cv2.inRange(img2, range6_lo, range6_hi)
    mask_range7 = cv2.inRange(img2, range7_lo, range7_hi)
    mask_range8 = cv2.inRange(img2, range8_lo, range8_hi)
    
    ash_mask = mask_range1+mask_range2+mask_range3+mask_range4+mask_range5+mask_range6+mask_range7+mask_range8
    ash_result = cv2.bitwise_and(img2, img2, mask=ash_mask)
    
    img[ash_mask>0]=(0,0,255)
    cv2.imwrite('Ash_Overlay_' + f[i] + '.tif', img)
    
    pixel_count = cv2.countNonZero(ash_mask)
    ash_pixels.append(pixel_count)
    
    dates = f[i][4:8]
    timeavg.append(dates)
    
    i = i + 1
    
fig3 = plt.figure(figsize=(10,10))
ax2 = fig3.add_subplot(111)
plt.plot(timeavg, ash_pixels, color = 'blue', linewidth = 4)
plt.title("Ash Coverage Over Time", fontsize=18, fontweight='bold')
plt.xlabel("Start Date & End Date", fontsize=14)
plt.ylabel("# of Pixels", fontsize=14)
fig3.savefig("Ash_Coverage_OverTime.png", dpi=200, bbox_inches='tight', pad_inches=0.7)
plt.close()

In [ ]:
sp7 = os.getcwd()
sf7 = os.listdir(sp7)

for file in sf7:
    if file.startswith('Ash'):
        shutil.move(os.path.join(sp7,file), os.path.join(sp2,file))

In [ ]:
os.chdir(sp2)
os.mkdir('Segmentation_Ash')

In [ ]:
sp8 = os.getcwd()
sf8 = os.listdir(sp8)
dp8 = sp2 + '\\Segmentation_Ash'

for file in sf8:
    if file.startswith('Ash'):
        shutil.move(os.path.join(sp8,file), os.path.join(dp8,file))